############# Markdown note ##################

<div class="alert alert-block alert-info"> <b>NOTE</b> Use blue boxes for Tips and notes. </div>

<div class="alert alert-block alert-success"> Use green boxes sparingly, and only for some specific purpose that the other boxes can't cover. For example, if you have a lot of related content to link to, maybe you decide to use green boxes for related links from each section of a notebook. </div>

<div class="alert alert-block alert-warning"> Use yellow boxes for examples that are not inside code cells, or use for mathematical formulas if needed. </div>

<div class="alert alert-block alert-danger"> In general, just avoid the red boxes. </div>

<img src="<path>" width=20% style="margin-left:auto; margin-right:auto">
<img src="<path>" width=40% style="float: right;">  

In [ ]:
%%sh

# reset all programs
rm -rf debug*

# MPI Advanced

Advanced concept of **Message Passing Interface** (MPI)

## Advanced DataTypes

## Previously on DataTypes...

DataTypes can be created with different MPI routines, for example:

* `MPI_Pack`: https://www.open-mpi.org/doc/v4.1/man3/MPI_Pack.3.php
* `MPI_Type_create_struct`: https://www.open-mpi.org/doc/v4.1/man3/MPI_Type_create_struct.3.php

<div class="alert alert-block alert-success"> Before using a new DataType, we shall <b>commit</b> it. </div>

* `MPI_Type_commit`: https://www.open-mpi.org/doc/v4.1/man3/MPI_Type_commit.3.php

<div class="alert alert-block alert-danger"> <b>REMARK</b>: Once a new data type is created we shall <b>destroy</b> it before closing the application:</div>

* `MPI_Type_free`: https://www.open-mpi.org/doc/v4.1/man3/MPI_Type_free.3.php

## Type_Contiguous

Simplest constructor. Makes count **copies** of an existing datatype

* `MPI_Type_contiguous`: https://www.open-mpi.org/doc/v4.1/man3/MPI_Type_contiguous.3.php

<img src="./Images/contigous.png" width=80% style="margin-left:auto; margin-right:auto">

## Type_Vector

Like contiguous, but allows for regular gaps (**stride**) in the displacements. 

* `MPI_Type_vector`, `MPI_Type_hvector`: https://www.open-mpi.org/doc/v4.1/man3/MPI_Type_vector.3.php

<img src="./Images/vector.png" width=60% style="margin-left:auto; margin-right:auto">

<div class="alert alert-block alert-info"> <b>NOTE</b>: For <code>MPI_Type_hvector</code> the stride is specified in bytes.. </div>

## Type_Index

An array of **non regular displacements** of the input data type is provided as the map for the new data type.

* `MPI_Type_indexed`, `MPI_Type_hindexed`: https://www.open-mpi.org/doc/v4.1/man3/MPI_Type_indexed.3.php

<img src="./Images/indexed.png" width=60% style="margin-left:auto; margin-right:auto">

<div class="alert alert-block alert-info"> <b>NOTE</b>: For <code>MPI_Type_hindexed</code> offsets are specified in byte. </div>

In [16]:
%%writefile main_vector.cpp

#include <iostream>
#include <mpi.h>

int main(int argc, char **argv) 
{
    MPI_Init(&argc, &argv);
    
    int rank;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    
    MPI_Status status;
    double a[4][4];
    for (unsigned int i = 0; i < 4; i++)
        for (unsigned int j = 0; j < 4; j++)
            a[i][j] = (rank == 0) ? i * 4.0 + j : 0.0;
    
    MPI_Datatype rowType, colType;
    
    MPI_Type_contiguous(4, MPI_DOUBLE, &rowType);
    MPI_Type_vector(4, 2, 4, MPI_DOUBLE, &colType);
    
    MPI_Type_commit(&rowType);
    MPI_Type_commit(&colType);
    
    if (rank == 0)
    {
        MPI_Send(&a[2][0], 1, rowType, 1, 10, MPI_COMM_WORLD);
        MPI_Send(&a[0][2], 1, colType, 1, 11, MPI_COMM_WORLD);
    }
    else if (rank == 1)
    {
        MPI_Recv(&a[2][0], 1, rowType, 0, 10, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
        MPI_Recv(&a[0][2], 1, colType, 0, 11, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
    }
    
    std::cout<< "Process "<< rank<< ": "<< std::endl;
    for (unsigned int i = 0; i < 4; i++)
    {
        for (unsigned int j = 0; j < 4; j++)
            std::cout<< (j == 0 ? "" : ", ")<< a[i][j];
        std::cout<< std::endl;
    }
    
    MPI_Type_free(&rowType);
    MPI_Type_free(&colType);
    
    MPI_Finalize();
    return 0;
}

Overwriting main_vector.cpp


In [17]:
%%sh

# compile program
mkdir -p ./debug_vector
cd debug_vector
cmake -DSOURCES="main_vector.cpp" ..
make

-- Configuring done
-- Generating done
-- Build files have been written to: /home/geoscore/Desktop/GEO++/Courses/PhdMPI/4_MPI_Advanced/debug_vector
Consolidate compiler generated dependencies of target 4_MPI_Advanced
[ 50%] Building CXX object CMakeFiles/4_MPI_Advanced.dir/main_vector.cpp.o
[100%] Linking CXX executable 4_MPI_Advanced
[100%] Built target 4_MPI_Advanced


In [18]:
%%sh

# run program
cd debug_vector
mpirun -np 2 4_MPI_Advanced

Process 0: 
0, 1, 2, 3
4, 5, 6, 7
8, 9, 10, 11
12, 13, 14, 15
Process 1: 
0, 0, 2, 3
0, 0, 6, 7
8, 9, 10, 11
0, 0, 14, 15
